# Try Fourier Ptychography

by [Jose David Marroquin Toledo](https://github.com/josemarroquintoledo/)

This example notebook simulates the whole process of the Fourier Ptychography. From 2-5 steps, it simulates the for forward imaging process generating low-resolution images adquired under different incident angles with a LED matrix. From 6-8 steps, this retrieves the phase information, lost in the measuring process, to reconstructs a high-resolution image from the Lo-Res image set.

## 1 Import Functions from Other Jupyter Notebooks

In [ ]:
%run ../jupyter-modules/fwdimaging.ipynb
%run ../jupyter-modules/phaseretrieval.ipynb
%run ../jupyter-modules/fputilities.ipynb

## 2 Simulate the Sample

In [ ]:
# You will find a compressed image set (CC) in img directory
# (fp-imgs-cc.zip). Uncompress it or use your own and uncomment the
# lines below.
# AMPLITUDE_IMG = '../img/BobMacInnes_microscopy-043--fly-wing_flickr_1024x685.tif'
# PHASE_IMG = '../img/pabloaez_Valparaiso-054_flickr_4000x3000.png'
sample, hires_w_px, hires_h_px = sim_sample(AMPLITUDE_IMG, PHASE_IMG)

## 3 Simulate the LED Matrix

In [ ]:
LEDS_PER_ROW = 9  # Number of LEDs per row of the LED matrix.
LED_DISTANCE = 4  # Distance between LEDs in mm.

xy_lst = sim_sqr_grid(9, 4)  # List of lists of (x, y) tuples.

## 4 Generate the Incident Wave Vectors for the LED Matrix

In [ ]:
import matplotlib.pyplot as plt
SAMPLE_DISTANCE = 100  # Distance between in mm between the LED matrix
                       # and the sample.

wv_xy = gen_wave_vectors(xy_lst, SAMPLE_DISTANCE)

## 5 Simulate the Low-Res Images for Each Different Incident Angle

In [ ]:
WAVELENGTH = 0.63e-6  # In m.
CCD_PX = 2.75e-6  # The size of the employed CCD in m.
NA = 0.08  # Numerical aperture (NA) of the objective lens.
# Add the Lo-Res set's path and uncomment the line below. If that does
# not exist, the will be created.
# LORES_SET_PATH =

# Generate the coherent transfer function of the coherent imaging
# system.
cft, lores_w_px, lores_h_px, dkx, dky, kx, ky = gen_cft(np.array(wv_xy),
    WAVELENGTH, CCD_PX, NA, hires_w_px, hires_h_px)
# Simulate the Lo-Res image set.

n_leds = LEDS_PER_ROW ** 2  # Total number of LEDs in the LED matrix.
sim_lores_set(sample, hires_w_px, hires_h_px, n_leds, cft, lores_w_px,
              lores_h_px, dkx, dky, kx, ky, LORES_SET_PATH)

## 6 Set a Reconstruction Sequence

In [ ]:
# Generate a sequence that starts in the center of the Fourier spectrum
# (central LED).
seq = get_matrix_seq(LEDS_PER_ROW)

## 7 Import the Lo-Res Image Set

In [ ]:
lores_set = read_lores_img_set(LORES_SET_PATH, n_leds, format='TIF')

## 8 Generate/Reconstruct the Hi-Res Image from the Lo-Res Image Set

In [ ]:
# Set a path for the reconstruction image an uncomment the line below.
OUTPUT_PATH = LORES_SET_PATH

gen_hires_img(lores_set, seq, hires_w_px, hires_h_px, cft, lores_w_px,
              lores_h_px, dkx, dky, kx, ky, OUTPUT_PATH)